## 0. Environment Setup
We create a smiliar environemnt that simulate the inputs and variables as per in ciop jobs

In [81]:
input1='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=230286758E25FD953AF736DE85F01CDF9C539EC7' #niger-hype-model.zip, one file when unzipped
input2='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9345ED73B72F49E6FF31B07B57013BC519210E24' #niger-hype-model-2.23.zip, one dir when unzipped
#input3='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9CE420998D9EC367470EEAEEA6B6DA0B0E4604D2' #niger-hype-data.zip, dirs when unzipped
TMPDIR="/workspace/tmpdir"
TMPDIR

[1] "/workspace/tmpdir"

1. Read input model
This inital step collect via proper opensearch call and copy the HYPE model config archive

In [82]:
app.date = format(Sys.time(), "%Y%m%d_%H%M")

## set application name
app.name = "forecast"
## ------------------------------------------------------------------------------
## flag which environment is used, if not set
if(!exists("app.sys")){
    app.sys ="tep"
    }
## ------------------------------------------------------------------------------
## load rciop package and set working directory to TMPDIR when running on TEP 
if(app.sys=="tep"){
    library("rciop")

    rciop.log ("DEBUG", " *** hypeapps-forecast *** TEP hydrological modelling applications ***", "/node_forecast/run.R")
    rciop.log ("DEBUG", " rciop library loaded", "/node_forecast/run.R")

    setwd(TMPDIR)
    rciop.log("DEBUG", paste(" R session working directory set to ",TMPDIR,sep=""), "/node_forecast/run.R")
}

Some rciop functions may need to be overridden in the notebook environment.
Set to your current needs.

In [83]:
ENABLE_STUB_RCIOP_LOG <- FALSE
#ENABLE_STUB_RCIOP_LOG <- TRUE

if (ENABLE_STUB_RCIOP_LOG == TRUE) {
    message("rciop.log() overridden by local stub function...")
    
    rciop.log <- function(type, message, proc='') {

        type <- type
        msg <- message
    if (proc=='') { proc <- "user process" }

    out.msg <- paste(format(Sys.time(), format="%Y-%m-%dT%H:%M:%S"), "[", type, "][", proc, "] ", msg, sep="")

    message(out.msg)
    } # rciop.log

} # end stub

In [84]:
#ENABLE_STUB_RCIOP_GETPARAM <- FALSE
ENABLE_STUB_RCIOP_GETPARAM <- TRUE

if (ENABLE_STUB_RCIOP_GETPARAM == TRUE) {
    message("rciop.getparam() overridden by local stub function...")
    # Change according to your test objective.
    # Would normally return values defined in application.xml for specified ids'.
    
    rciop.getparam <- function(id) {
        value <- NULL # or ""
        
        if (id == "model") {
            value <- "https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=230286758E25FD953AF736DE85F01CDF9C539EC7"
        }
        else if (id == "idate" ) {
            value <- "2018-09-01"
        }
        else {
            message("rciop.get_param:id not found...")
        }
        
        return (value)
    } # rciop.getparam

} # end stub

rciop.getparam() overridden by local stub function...


Add your code to test...
Remove current, or paste other code or add additional code blocks.

In [85]:
## Read the main input
## This is the reference link to the model configuration

# print
rciop.log("INFO", paste("Processing input:", input1, sep=" "))

# Query the input reference
opensearchCmd=paste("opensearch-client '",input1,"' enclosure")
input_enclosure <- system(command = opensearchCmd,intern = T)
rciop.log("INFO", input_enclosure)

# Download the file
model_file <- rciop.copy(input_enclosure, TMPDIR, uncompress=TRUE)

if (model_file$exit.code==0) {
    local.url <- model_file$output
}

# Works for single file after unzip
local.url # One file, complete path incl. filename
my_data <- read.delim(local.url)
print (my_data)

[1] " -o /workspace/tmpdir https://store.terradue.com/hydro-smhi/models/niger-hype-model.zip"


[1] "/workspace/tmpdir/info-forecast.txt"

                                         X...Template.info.txt.for.Niger.HYPE.
1                                                                           !!
2                  !!    - 10 day determniistic forecast issue date 2017-01-01
3                 !!    - model version 2.23, hype version 5.x.0 (assimiltion)
4                                                                           !!
5                !! This file is part of Hydrology TEP hypeapps applications. 
6                                                                           !!
7          !! ----------------------------------------------------------------
8             !! bdate=cdate= first date of the forecast (forecast issue date)
9             !! -------------------------------------------------------------
10                                                            bdate 2017-01-01
11                                                            cdate 2017-01-01
12                              !! -----------------

In [86]:
par_model <- rciop.getparam("model")
message(par_model)

https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=230286758E25FD953AF736DE85F01CDF9C539EC7


In [87]:
## Read the main input
## This is the reference link to the model configuration

# print
rciop.log("INFO", paste("Processing input:", input2, sep=" "))

# Query the input reference
opensearchCmd=paste("opensearch-client '",input2,"' enclosure")
input_enclosure <- system(command = opensearchCmd,intern = T)
rciop.log("INFO", input_enclosure)

# Download the dir(s)
model_config_dir <- rciop.copy(input_enclosure, TMPDIR, uncompress=TRUE)

if (model_config_dir$exit.code==0) {
    local.dir <- model_config_dir$output
}
local.dir # File or directory, complete path incl.
#if (local.dir == "" || local.dir == NULL) {
#    message("No model config path...")
#    q(99)
#}

model_config_file <- "dependencies.txt" # Shall be parsed to get different config items
#hype_file <- "info-forecast.txt"

path_to_file <- paste(local.dir, model_config_file, sep="/")
path_to_file
# Read contents of file, handling separators for now. model_config_data will probably be a matrix, 3 columns for now
model_config_data <- read.csv2(path_to_file, header=FALSE, sep=";")
#class(model_config_data) # data.frame
#dim(model_config_data) # 6,3 (rows,cols)
# Add column names
names(model_config_data) <- c('subdir','url','querypattern') # Now works to use as index, colnamnes maybe only on type matrix
model_config_data
# Select column by name
#model_config_data[,'query-pattern'] # all rows, col 3 ok
#model_config_data[,c('subdir','querypattern')] # all rows, multiple columns ok

#model_config_data$subdir
#model_config_data[3,'subdir']
# Iterate over each row to locate e.g. sub directory, od-daily, to use column data
for (r in 1:nrow(model_config_data)) { #  if statement below not works
    subdir <- model_config_data[r,'subdir']
    if (subdir == 'od-daily') {
        message(paste0("Found od-daily at row index:", r))
    }
}

[1] " -o /workspace/tmpdir https://store.terradue.com/hydro-smhi/models/niger-hype-model-2.23.zip"


[1] "/workspace/tmpdir/niger-hype-model-2.23"

[1] "/workspace/tmpdir/niger-hype-model-2.23/dependencies.txt"

subdir,url,querypattern
hydrogfdei,https://catalog.terradue.com/hydro-smhi/description,"[hydrogfdei,SMHI]"
od,https://catalog.terradue.com/hydro-smhi/description,"[od,SMHI]"
ei-monthly,https://catalog.terradue.com/hydro-smhi/description,"[ei,monthly,SMHI]"
od-daily,https://catalog.terradue.com/hydro-smhi/description,"[od,daily,SMHI]"
od-monthly,https://catalog.terradue.com/hydro-smhi/description,"[od,montly,SMHI]"
ecoper,https://catalog.terradue.com/hydro-smhi/description,"[ecoper,SMHI]"


Found od-daily at row index:4


In [88]:
# opensearch for netcdf files:
